In [1]:
from os import environ
import pandas as pd
import numpy as np

ImportError: No module named 'pandas'

In [ ]:
def open_excel(filename, idx_sheet=0):
    xl = pd.ExcelFile(filename)
    df = xl.parse(xl.sheet_names[idx_sheet])
    assert 'Subject' in df.columns
    assert 'Electrode' in df.columns
    assert 'pdur' in df.columns
    assert 'type' in df.columns
    assert 'data' in df.columns
    df.drop(df.index, inplace=True)
    return df

In [ ]:
def load_row(df, subject, electrode, pdur, dtype='truth'):
    rows = (df['Subject'] == subject) & (df['Electrode'] == electrode) & \
           (df['type'] == dtype) & (df['pdur'] == pdur)
    if not np.any(rows):
        return None, None
    elif np.count_nonzero(rows) > 1:
        print('More than one row found:')
        print(df.loc[rows, :])
        return None, None

    # Data points: [all pdurs, all amps]
    data = np.array(df.loc[rows, 'data':])
    data = data.reshape((2, -1))
    return data[0, :], data[1, :]

In [ ]:
def run_fixed_duration(subjects, electrodes, pdurs):
    df = open_excel(environ['SECOND_SIGHT_DATA'] + '/horsager2009/fixed-duration.xlsx')
    
    for s in subjects:
        for e in electrodes:
            for p in pdurs:
                x, y = load_row(df, s, e, p, dtype='truth')
                if not x and not y:
                    print('No row of subject %s, electrode %s, pdur %f' % (s, e, p))
                    continue

In [ ]:
run_fixed_duration('S04', 'A1', 10)